1.: **Készítsen lekérdezést, amely sorszámozza a vendégeket a foglalások száma alapján**

- A sorszám mellett jelenjen meg a vendég azonosítója, neve és a foglalások száma is
    
- A legtöbbet foglaló vendég kapja az egyes sorszámot
    
- Holtverseny esetén a számozás annyit ugorjon, ahány egyforma érték van

In [ ]:
SELECT RANK() OVER( order by count(f.FOGLALAS_PK) desc ) as 'helyezes',
       v.USERNEV,
       v.NEV,
       count(f.FOGLALAS_PK)  
from Vendeg v JOIN Foglalas f ON v.USERNEV=f.UGYFEL_FK
GROUP BY v.USERNEV, v.nev
ORDER BY 'helyezes'

2.: **Listázzuk azon vendégek nevét, email-címét és felhasználói nevét, akik egynél többször foglaltak!**

- Hagyjuk ki azokat a vendégeket, akik május hónapban születtek!

In [ ]:
SELECT v.USERNEV,
       v.NEV, 
       v.EMAIL
From Vendeg v JOIN Foglalas f ON v.USERNEV=f.UGYFEL_FK
where month(v.SZUL_DAT)<>5
GROUP BY v.USERNEV, v.NEV, v.EMAIL
HAVING count(f.FOGLALAS_PK)>1

**3\. Listázzuk azon vendégek adatait, akik a legtöbb férőhelyes szobát (vagy szobákat) már lefoglalták!**

- Csak klímás szoba jöhet számításba

In [ ]:
select v.USERNEV,
       v.NEV,
       v.EMAIL,
       v.SZUL_DAT,
       v.SZAML_CIM
From Foglalas f JOIN Vendeg v ON f.UGYFEL_FK=v.USERNEV
              JOIN Szoba sz ON f.SZOBA_FK=sz.SZOBA_ID
WHERE sz.FEROHELY in (
    select max(FEROHELY)
    from szoba
    where KLIMAS='i'
)

4. **Hozza létre REDIS Cloud parancssorban a következő táblákat:**

- **IRO**(ID, nev, nemzetiseg)
    
- **KONYV**(ID, cim, mufaj, IRO\_ID)
    

**Vigyen fel táblánként legalább 2–2 sornyi adatot!**

HSET IRO:1 id 1 nev "Petofi Sandor" nemzetiseg "magyar"

HSET IRO:2 id 2 nev "Kaffka nemzetiseg "cseh"

  

HSET KONYV:1 id 1 cim "Az arany ember" mufaj "regeny" iro\_id 1

HSET KONYV:2 id 2 cim "Az atvaltozas" mufaj "novella" iro\_id 2

<span style="color: var(--vscode-foreground);"><b>5: A Neo4j Sandbox-ban vagy a Desktopban jelenítsük meg azon személyeket, akik</b> </span> **nem szerepeltek egy filmben Tom Hanks-szel**<span style="color: var(--vscode-foreground);">.</span>

- Egy szereplő neve csak egyszer jelenjen meg.

MATCH (p:Person)

<span style="color: var(--vscode-foreground);">WHERE NOT EXISTS {</span>

  MATCH (p)-\[:ACTED\_IN\]-\>(f:Movie)\<-\[:ACTED\_IN\]-(th:Person {name: "Tom Hanks"})

}

RETURN DISTINCT p.nam

**6: <span style="background-color: rgba(127, 127, 127, 0.1); color: var(--vscode-foreground);">A MongoDB Atlas </span> `sample_mflix` <span style="background-color: rgba(127, 127, 127, 0.1); color: var(--vscode-foreground);">adatbázis </span> `comments` <span style="background-color: rgba(127, 127, 127, 0.1); color: var(--vscode-foreground);">gyűjteményéből kérdezzük le, hogy </span> mely felhasználók (email) <span style="background-color: rgba(127, 127, 127, 0.1); color: var(--vscode-foreground);">írtak </span> 10-nél több kommentet<span style="background-color: rgba(127, 127, 127, 0.1); color: var(--vscode-foreground);">?</span>**

db.comments.aggregate(\[

  {

    $group: {

      \_id: "$email",

      comment\_count: { $sum: 1 }

    }

  },

  {

    $match: {

      comment\_count: { $gt: 10 }

    }

  },

  {

    $project: {

      \_id: 0,

      email: "$\_id",

      comment\_count: 1

    }

  }

\])

**7.: A DataStax Astra minta adatai közül a `movies_and_tv` táblájából kérdezzük le, hogy melyik 10 film kapott a legjobb értékelést 2010-ben!  
Csak a filmek címe jelenjen meg!**

use movies;

  

SELECT title 

FROM movies\_and\_tv 

WHERE release\_year = 2010 

ORDER BY rating DESC 

LIMIT 10;